# 영업 성공 여부 분류 경진대회
## 팀 : LG영업왕 (김수민, 김준형, 김현동, 정찬혁)

## 1. 데이터 확인

### 필수 라이브러리

In [ ]:
!pip uninstall googletrans -y
!pip install googletrans==3.1.0a0
!pip install catboost

Found existing installation: googletrans 3.1.0a0
Uninstalling googletrans-3.1.0a0:
  Successfully uninstalled googletrans-3.1.0a0
Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-3.1.0a0-py3-none-any.whl

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import re
import pandas as pd
import numpy as np

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

from googletrans import Translator
translator = Translator()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    np.random.seed(seed) #numpy를 사용할 경우 고정
seed_everything(42)

### 데이터 셋 읽어오기

In [ ]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [ ]:
df_train[:10]

,bant_submit,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_from_channel,lead_description,lead_desc_length,event_name,inquiry_type,prefer_ver_count,prefer_ver_mean,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,transfer_agreement,ver_cus,ver_pro,ver_win_rate_x,ver_win_rate_mean_upper,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,0.75,NaN,/DKI Jakarta/Indonesia,AS,NaN,33931,2022-10-15,End-Customer,Existing,Enterprise,4.0,NaN,NaN,NaN,engineering,lg.com(local),NaN,30,AS_IN_LG.com_I2B_2022,Quotation or purchase consultation,0.076923,0.019648,vrf,NaN,NaN,/DKI Jakarta/Indonesia,director,LGEIN,less than 3 months,Y,0,0,0.000060,1.0,0.128571,hospital & health care,General Hospital,870,False
1,0.25,NaN,/Minneola/United States,ID,0.390244,42490,2023-05-31,NaN,New,SMB,NaN,NaN,NaN,NaN,NaN,promotion page,Are there any grants for technology available?,46,ID_US_Createboard_Campaign,Quotation or Purchase Consultation,0.095436,0.067718,NaN,Digital Signage or Commercial TVs,NaN,/Minneola/United States,none,LGEUS,NaN,N,0,0,0.000572,0.0,0.048630,education,NaN,501,True
2,0.25,NaN,4880 Stevens Creek Blvd Cloud IT/Hollywood/Un...,IT,NaN,10350,2023-04-19,Solution Eco-Partner,New,SMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,IT_US_Cloud_IT_VMUG_UserCon_Event_Leads_230323,Quotation or Purchase Consultation,NaN,NaN,NaN,NaN,NaN,4880 Stevens Creek Blvd Cloud IT/Hollywood/Un...,none,LGEUS,NaN,Y,0,0,NaN,NaN,NaN,NaN,NaN,375,True
3,1.00,"$100,000 ~ $500,000",/Wiesbaden/Germany,ID,0.017544,19750,2021-10-30,NaN,New,SMB,NaN,NaN,NaN,NaN,education,lg.com,"Dear Ladies and Gentlemen, \n\nfor three of ou...",232,ID_HQ_LG.com_I2B,Quotation or Purchase Consultation,0.095436,0.067718,oled signage,NaN,NaN,/Wiesbaden/Germany,manager,LGEDG,less than 3 months,Y,0,0,0.000572,0.0,0.048630,education,NaN,359,False
4,0.50,NaN,//Singapore,ID,NaN,45802,2023-04-11,Specifier/ Influencer,New,SMB,NaN,NaN,NaN,NaN,marketing,lg.com(local),test,4,ID_SL_LG.com_I2B_Dev,Quotation or purchase consultation,NaN,NaN,video wall signage,NaN,NaN,//Singapore,none,LGESL,less than 3 months,Y,0,0,NaN,NaN,NaN,NaN,NaN,198,False
5,1.00,"$50,000 ~ $100,000",/Dallas/India,ID,0.028926,6837,2022-11-08,Service Partner,New,SMB,NaN,NaN,NaN,NaN,engineering,lg.com(local),"We are AV integrator, client has a Planar TV V...",135,ID_IL_LG.com_I2B,Quotation or purchase consultation,0.145228,0.103511,oled signage,Curvable OLED,55EF5G- L,/Dallas/India,associate/analyst,LGEIL,less than 3 months,Y,0,1,0.001183,0.0,0.049840,retail,NaN,174,False
6,1.00,"More than $1,000,000",//Spain,ID,NaN,47466,2022-05-12,NaN,Existing,Enterprise,45.0,NaN,NaN,NaN,arts and design,lg.com(global),Hello,5,NaN,Technical Consultation,NaN,NaN,oled signage,NaN,NaN,//Spain,c-level executive,LGEES,less than 3 months,Y,0,0,NaN,NaN,NaN,NaN,NaN,196,False
7,0.75,NaN,/New Delhi/India,ID,NaN,47466,2022-04-17,NaN,Existing,Enterprise,4.0,NaN,NaN,NaN,NaN,facebook,Aryan study success point,25,ID_IL_Social BANT Camp_220308,Quotation or purchase consultation,NaN,NaN,interactive digital board,NaN,NaN,/New Delhi/India,ceo/founder,LGEIL,less than 3 months,N,0,0,NaN,NaN,NaN,NaN,NaN,175,False
8,0.25,NaN,2042 Wooddale Dr Ste 250 Commercial Display/S...,ID,0.044643,22031,2023-03-01,End-Customer,New,SMB,NaN,1.0,NaN,1.0,general management,NaN,NaN,3,ID_US_ISE_2023_Event_Leads_230227,Quotation or Purchase Consultation,0.211618,0.027756,NaN,NaN,NaN,2042 Wooddale Dr Ste 250 Commercial Display/S...,none,LGEUS,NaN,Y,1,0,0.003079,0.0,0.064566,corporate / office,NaN,194,False
9,1.00,"$200,000 ~ $400,000",/Florence/Italy,AS,NaN,24755,2023-01-10,End Customer,New,SMB,NaN,NaN,NaN,NaN,others,lg.com(global),NaN,489,AS_Global_Europe_LG.com_I2B,Quotation or Purchase Consultation,NaN,NaN,vrf,NaN,NaN,/Florence/It

### 결측치 변수 및 결측률 파악

In [ ]:
mis_val = df_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

/tmp/ipykernel_4969/1061145631.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299


,mis_val,mis_val_bool,ratio
transfer_agreement,530,True,0.008938
inquiry_type,941,True,0.015869
customer_country,982,True,0.016560
customer_country.1,982,True,0.016560
event_name,1132,True,0.019090
lead_from_channel,3295,True,0.055566
customer_job,18733,True,0.315908
product_category,19374,True,0.326717
lead_description,30203,True,0.509334
expected_timeline,30863,True,0.520464


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   expected_budget          24122 non-null  object 
 2   customer_country         58317 non-null  object 
 3   business_unit            59299 non-null  object 
 4   com_reg_ver_win_rate     14568 non-null  float64
 5   customer_idx             59299 non-null  int64  
 6   lead_date                59299 non-null  object 
 7   customer_type            15338 non-null  object 
 8   customer_history         59299 non-null  object 
 9   enterprise               59299 non-null  object 
 10  historical_existing_cnt  13756 non-null  float64
 11  id_strategic_ver         3444 non-null   float64
 12  it_strategic_ver         1121 non-null   float64
 13  idit_strategic_ver       4565 non-null   float64
 14  customer_job          

## 전처리

### customer_country.1 drop

In [ ]:
# customer_country , customer_country.1의 값이 일치함.
(df_train['customer_country'].fillna(0) == df_train['customer_country.1'].fillna(0)).value_counts()

True    59299
Name: count, dtype: int64

In [ ]:
pre_train = df_train.drop(labels = ['customer_country.1'],axis=1)
pre_test = df_test.drop(labels = ['customer_country.1','id'],axis=1)

### 중요도 낮은 feature drop

In [ ]:
pre_train.drop(['ver_cus','ver_pro','idit_strategic_ver','it_strategic_ver',
                'id_strategic_ver','bant_submit','lead_description','ver_win_rate_mean_upper'], axis=1, inplace=True)
pre_test.drop(['ver_cus','ver_pro','idit_strategic_ver','it_strategic_ver',
               'id_strategic_ver','bant_submit','lead_description','ver_win_rate_mean_upper'], axis=1, inplace=True)

### 수치형/범주형 컬럼 분류

['it_strategic_ver','id_strategic_ver','idit_strategic_ver','lead_owner','ver_cus','ver_pro','customer_idx']
: float -> 범주형 데이터로 변환

In [ ]:
dtypes = pre_train.dtypes

# 각 열의 고유값 개수 확인
unique_value_counts = pre_train.nunique()

# 수치형 변수가 아닌 컬럼을 저장할 리스트 초기화
object_columns = []

# 각 열에 대해 데이터 타입이 'object'이거나 고유값 개수가 일정 이하인 경우를 확인하여 non_numeric_columns 리스트에 추가
for col in pre_train.columns:
    if dtypes[col] == 'object':
        object_columns.append(col)

# 수치형 변수가 아닌 컬럼 출력
print("범주형 컬럼:", object_columns,"\n")
print("수치형 컬럼 :", list(set(pre_train.columns)-set(object_columns)))

범주형 컬럼: ['expected_budget', 'customer_country', 'business_unit', 'lead_date', 'customer_type', 'customer_history', 'enterprise', 'customer_job', 'lead_from_channel', 'event_name', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'transfer_agreement', 'business_area', 'business_subarea'] 

수치형 컬럼 : ['com_reg_ver_win_rate', 'ver_win_rate_x', 'is_converted', 'customer_idx', 'prefer_ver_count', 'lead_owner', 'ver_win_ratio_per_bu', 'lead_desc_length', 'historical_existing_cnt', 'prefer_ver_mean']


In [ ]:
# 범주형으로 변경
columns_categorized = ['lead_owner','customer_idx']

for n in columns_categorized:
    pre_train[n] = pre_train[n].fillna(0)
    pre_train[n] = pre_train[n].astype('int').astype('object')

### 영어로 번역해서 언어 통일

customer_job , inquiry_type , product_category , product_subcategory , product_modelname

#### 번역 test code

In [ ]:
translator = Translator()

def remove_special_characters(text):
    return re.sub(r'\W+', ' ', text)

def is_english(sentence):
    if sentence is np.nan:
        return sentence

    non_alpha_chars = re.sub(r'\W+', '', sentence)
    print(non_alpha_chars)
    non_alpha_chars = re.search(r'[^a-zA-Z]', non_alpha_chars)
    print(non_alpha_chars)

    if non_alpha_chars:
        changed_chars = translator.translate(sentence,dest='en').text
#         return True
    else:
        changed_chars = sentence

    return changed_chars

# 특수 기호가 포함된 문자열 예시
n = np.nan
text_with_special_characters_1 = 'systèmes de débit à réfrigérant variable (drv)'
text_with_special_characters_2 = '醫院電視'
text_with_special_characters_3 = 'refrigerator,built-in/cooking'

# 특수 기호를 제거한 문자열 출력
print(is_english(text_with_special_characters_1))

systèmesdedébitàréfrigérantvariabledrv
<re.Match object; span=(4, 5), match='è'>
variable refrigerant flow (drv) systems


#### 번역 전처리

In [ ]:
def en_dic(df,col):
    unique_value = df[col].unique()
    trans_dic = {}

    for sen in unique_value:
        if sen is np.nan:
            trans_dic[sen] = sen
            continue

        non_alpha_chars = re.sub(r'\W+', '', sen)
        non_alpha_chars = re.search(r'[^a-zA-Z0-9]', non_alpha_chars)

        if non_alpha_chars:
            changed_chars = translator.translate(sen, dest='en').text
            trans_dic[sen] = changed_chars

    return trans_dic

def to_english(df,col):
    # en_dic에 있는 값이면 해당 행 값을 사전의 값으로 교체
    df[col] = df[col].apply(lambda x: dic[x] if x in dic else x)

In [ ]:
need_to_en = ['customer_job' , 'inquiry_type' , 'product_category' , 'product_subcategory' , 'product_modelname']
for n in need_to_en:
    dic = en_dic(pre_train,n)
    to_english(pre_train,n)

for n in need_to_en:
    dic = en_dic(pre_test,n)
    to_english(pre_test,n)

### // === 본선 추가 내용 ===

In [ ]:
def preprocess_channel(channel):
    if pd.isna(channel):
        return np.nan

    if 'webinar' in channel:
        return 'webinar'
    elif 'sem' in channel:
        return 'seminar'
    elif  channel == 'micrositio':
        return 'microsite'
    elif channel == 'lg.co':
        return 'lg.com'
    elif 'newsletter' in channel:
        return 'newsletter'
    elif 'exhibit' in channel:
        return 'exhibition'
    else:
        return channel

In [ ]:
pre_train['lead_from_channel'].value_counts()

lead_from_channel
lg.com(local)                              16247
facebook                                   14138
lg.com                                      5748
webinar                                     5291
lg.com(global)                              3043
webinar(online)                             1968
microsite                                   1563
youtube                                     1518
c-display                                   1219
linkedin                                    1188
seminar                                     1084
brochure & leaflet                           943
etc.                                         412
newsletter(+blogsubscription)                396
roadshow                                     337
exhibition                                   308
dealertrip                                   173
instagram                                     81
sem                                           55
promotion page                                49
di

In [ ]:
pre_train['lead_from_channel'] = pre_train['lead_from_channel'].apply(preprocess_channel)
pre_test['lead_from_channel'] = pre_test['lead_from_channel'].apply(preprocess_channel)

In [ ]:
etc_channel_list = pre_train['lead_from_channel'].apply(preprocess_channel).value_counts().loc[lambda x : x < 10].index.to_list()

def channel_etc(channel):
    if pd.isna(channel):
        return np.nan

    if channel in etc_channel_list:
        return 'etc.'
    else:
        return channel

In [ ]:
pre_train['lead_from_channel'].apply(channel_etc).value_counts().iloc[:100]

lead_from_channel
lg.com(local)         16247
facebook              14138
webinar                7259
lg.com                 5750
lg.com(global)         3043
microsite              1570
youtube                1518
c-display              1219
linkedin               1188
seminar                1146
brochure & leaflet      943
etc.                    472
newsletter              403
roadshow                337
exhibition              310
dealertrip              173
instagram                81
promotion page           49
digital showroom         39
conference               31
website                  29
gdn                      25
sub(webpage)             20
db(imported)             14
Name: count, dtype: int64

In [ ]:
pre_train['lead_from_channel'] = pre_train['lead_from_channel'].apply(channel_etc)

### // ==================

### lead_date

In [ ]:
pre_train['lead_date_y'] = pd.to_datetime(pre_train['lead_date']).dt.year.astype('object').fillna(" ")
pre_train['lead_date_m'] = pd.to_datetime(pre_train['lead_date']).dt.month.astype('object').fillna(" ")
pre_train['lead_date_dow'] = pd.to_datetime(pre_train['lead_date']).dt.dayofweek.astype('object').fillna(" ")

pre_test['lead_date_y'] = pd.to_datetime(pre_test['lead_date']).dt.year.astype('object').fillna(" ")
pre_test['lead_date_m'] = pd.to_datetime(pre_test['lead_date']).dt.month.astype('object').fillna(" ")
pre_test['lead_date_dow'] = pd.to_datetime(pre_test['lead_date']).dt.dayofweek.astype('object').fillna(" ")

### customer_country : 국가에 해당하는 단어 추출

In [ ]:
import string
import re

def remove_punctuation(text):
    if pd.isna(text):
        return np.nan

    text = text.lower()
    text = re.sub('[~_-]', ' ', text)
    text = re.sub('[&(]', ' ', text)
    text = re.sub('[).]', ' ', text)
    text = text.rstrip()
    text = text.lstrip()
    text = re.sub('\s\s', ' ', text)
    text = re.sub('\s\s', ' ', text)  # 띄어쓰기 연속 제거

    return text

def remove_slash(text):
    if pd.isna(text):
        return np.nan
    return re.sub('/', ' ', text)

In [ ]:
# 데이터 통일성을 위해 소문자 변경, 공백 축소
def preprocess_str(col):
    pre_train[col] = pre_train[col].str.lower()
    pre_test[col] = pre_test[col].str.lower()

    mask_train = pre_train[col].notnull()
    mask_test = pre_test[col].notnull()
    pre_train.loc[mask_train, col] = pre_train.loc[mask_train, col].apply(lambda x: x.replace(" ", ""))
    pre_test.loc[mask_test, col] = pre_test.loc[mask_test, col].apply(lambda x: x.replace(" ", ""))

# '/'를 기준으로 customer_country에서 국가에 해당하는 단어 추출
def process_country(country):
    result = np.nan

    if pd.isna(country):
        return np.nan

    parts = [x for x in country.split('/') if x != '']

    if len(parts) >= 1:
        result = parts[-1]
    else:
        return np.nan

    if '@' in str(result):
        return np.nan

    return result

In [ ]:
print("전처리 전 nunique : ")
pre_train['customer_country'].nunique(), pre_test['customer_country'].nunique()

전처리 전 nunique : 


(15399, 7134)

In [ ]:
for n in object_columns:
    preprocess_str(n)

pre_train['customer_country'] = pre_train['customer_country'].apply(process_country)
pre_test['customer_country'] = pre_test['customer_country'].apply(process_country)
pre_train['customer_country'] = pre_train['customer_country'].replace("us", "unitedstates")
pre_test['customer_country'] = pre_test['customer_country'].replace("us", "unitedstates")

In [ ]:
print("전처리 후 nunique : ")
pre_train['customer_country'].nunique(), pre_test['customer_country'].nunique()

전처리 후 nunique : 


(1285, 277)

### ====================

### budget

In [ ]:
pre_train['expected_budget'] = pre_train['expected_budget'].apply(remove_punctuation)
pre_train['expected_budget'] = pre_train['expected_budget'].str.replace(',', '').str.strip()

pre_test['expected_budget'] = pre_test['expected_budget'].apply(remove_punctuation)
pre_test['expected_budget'] = pre_test['expected_budget'].str.replace(',', '').str.strip()

In [ ]:
currency = ['$','€','£']

# 화폐 단위를 포함하는 문자열인지 판단
def is_currency(text):
  if pd.isna(text):
      return np.nan

  for keyword in currency:
    if keyword in text:
      return text
    else:
      continue

  return np.nan

In [ ]:
pre_train['expected_budget'] = pre_train['expected_budget'].apply(is_currency)
pre_test['expected_budget'] = pre_test['expected_budget'].apply(is_currency)

In [ ]:
import re
import numpy as np

def process_budget(budget):
    # 부동 소수점 숫자인 경우 처리
    if isinstance(budget, float):
        return np.nan  # 부동 소수점 숫자이면 NaN 반환

    currencies = ['$', '€', '£']

    if '$' in budget:
        parts = [part.strip() for part in budget.split('$')]
    elif '€' in budget:
        parts = [part.strip() for part in budget.split('€')]
    elif '£' in budget:
        parts = [part.strip() for part in budget.split('£')]

    for i in range(len(parts)):
        parts[i] = parts[i].replace(' ', '')
        parts[i] = re.sub(r'\D', '', parts[i]) # 문자가 끼어들어와서 split되는 경우 숫자만 정제

    # A ~ B 의 꼴은 len(parts) = 3
    if len(parts) == 3:
        # 10 000€  50 000€와 같이 끝 문자에 단위가 있는 경우...
        if parts[2] =='':
          a = int(parts[0])
          b = int(parts[1])
          return (b+a)/2
        a = int(parts[1])
        b = int(parts[2])
        if (b+a)/2 < 20: # M단위로 표시된 것들 >> 1000000으로 통일
          return 1000000
        return (b+a)/2

    # less than(more than) A  꼴은 len(parts) = 2
    # A로 return
    elif len(parts) == 2:

        if(parts[1]==''):
          return np.nan
        a = int(parts[1])
        if a > 2000000: # 값이 비정상적으로 큰 경우
          return np.nan
        return a
    elif len(parts) == 5:
        if parts[2] =='':
            a = int(parts[0])
            b = int(parts[1])
            return (b+a)/2
        a = int(parts[1])
        b = int(parts[2])
        if (b+a)/2 < 10:
          return ((b+a)/2)*1000000
        return (b+a)/2
    else:
        print("check : ",parts)
        return np.nan

result = process_budget('$50 337 000cop   ')
print(result)

nan


In [ ]:
pre_train['expected_budget'] = pre_train['expected_budget'].apply(process_budget)
pre_test['expected_budget'] = pre_test['expected_budget'].apply(process_budget)

In [ ]:
budget_mode = pre_train['expected_budget'].value_counts().index[0]
budget_mode

100000.0

In [ ]:
pre_train['expected_budget'] = pre_train['expected_budget'].fillna(budget_mode)
pre_test['expected_budget'] = pre_test['expected_budget'].fillna(budget_mode)

In [ ]:
pre_train['expected_budget'].value_counts()

expected_budget
100000.0     42600
10000.0       5817
30000.0       3278
300000.0      2333
1000000.0     1869
75000.0       1343
50000.0        871
750000.0       708
150000.0       190
7000.0          81
700000.0        70
500000.0        69
900000.0        55
21000.0          6
52500.0          3
40000.0          2
123000.0         2
210000.0         1
250000.5         1
Name: count, dtype: int64

### event_name

In [ ]:
# 각각을 뽑아내기보단 unit과 같이 뽑는 게 나을 것 같음 >> 유의미한 데이터만 추출

def process_event(event):
    result = np.nan

    if pd.isna(event):
        return np.nan

    parts = [x.strip() for x in event.split('_')]  # 모든 요소를 포함하여 분할
    if len(parts) >= 1:
      blank=''
      if len(parts) == 4:
        filter_list = parts[:3]
        for i in filter_list:
          blank += ' '+i
        return blank

      elif len(parts) > 4:
        filter_list = parts[:4]
        for i in filter_list:
          blank += ' '+i
        return blank

    else:
        return np.nan


    return result
process_event('id_il_socialbantcamp_220308')

' id il socialbantcamp'

In [ ]:
pre_train['event_name'] = pre_train['event_name'].apply(process_event)
pre_test['event_name'] = pre_test['event_name'].apply(process_event)

### historical_existing_cnt

In [ ]:
pre_train['historical_existing_cnt'].fillna(-1, inplace=True)
pre_test['historical_existing_cnt'].fillna(-1, inplace=True)

### lead_date

### =================

### customer_job : 같은 직업으로 해석가능한 단어를 묶어서 처리

In [ ]:
# 묶을 수 있는 직업은 처리. 세분화 되어있더라도 모델에게는 아예 다른 직업으로 인식될 것.
def preprocess_job(job):
    if pd.isna(job):
        return np.nan

    if 'direct' in job:
        return 'director'
    elif 'consult' in job:
        return 'consultant'
    elif 'owner' in job:
        return 'owner'
    elif 'market' in job:
        return 'marketor'
    elif 'account' in job:
        return 'account'
    elif 'manage' in job:
        return 'manager'
    elif 'admin' in job:
        return 'administrator'
    elif 'design' in job:
        return 'designer'
    elif 'engineer' in job:
        return 'engineer'
    elif 'manager' in job:
        return 'manager'
    elif 'sales' in job:
        return 'sales'
    elif 'research' in job:
        return 'research'
    elif 'support' in job:
        return 'support'
    elif 'consultant' in job:
        return 'consultant'
    elif 'finance' in job:
        return 'finance'
    elif 'sale' in job:
        return 'sales'
    elif 'educat' or 'teach' in job:
        return 'educator'
    elif 'healthcare'in job:
        return 'healthcare'
    elif 'human' or 'hr posting' in job:
        return 'hr'
    elif 'inform' in job:
        return 'it'
    elif 'military' in job:
        return 'military'
    elif 'comunica' in job:
        return 'comunicator'
    else:
        return job

In [ ]:
# 전처리 전 'admin'을 포함하는 단어로 표현된 값
pre_train['customer_job'][pre_train['customer_job'].fillna('').str.contains('admin')].value_counts()

customer_job
administrative             3359
administration                8
administrativeassistant       3
pacsadministrator             2
admin                         2
networkadministrator          1
itadministrator               1
imagingadministrator          1
systemsadministrator          1
projectadministrator          1
itadmin                       1
adminassistant                1
facilityadministrator         1
platformadministrator         1
Name: count, dtype: int64

In [ ]:
pre_train['customer_job'].fillna(0)
pre_train['customer_job'] = pre_train['customer_job'].apply(preprocess_job)
pre_test['customer_job'] = pre_test['customer_job'].apply(preprocess_job)

In [ ]:
# 전처리 후
pre_train['customer_job'][pre_train['customer_job'].fillna('').str.contains('admin')].value_counts()

customer_job
administrator    3383
Name: count, dtype: int64

### other과 유사 단어를 other로 대체
'inquiry_type', 'customer_position', 'customer_type'

In [ ]:
# 'inquiry_type', 'customer_position', 'customer_type' 열에서 특정 문자열을 'other'로 대체
columns_to_replace_other = ['inquiry_type', 'customer_position', 'customer_type']
for col in columns_to_replace_other:
    pre_train[col] = pre_train[col].replace(['other','other_', 'etc.', 'others'], 'other')
    pre_test[col] = pre_test[col].replace(['other','other_', 'etc.', 'others'], 'other')

### product_category : 여러 카테고리를 가진 값을 , 기준으로 분해 -> 증강

In [ ]:
pre_train[pre_train['product_category'] == 'monior/monitortv,tv,commercialtv,pc,refrigerator,solar,rac,washingmachine,mobile,ess,audio/video']

,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,customer_job,lead_from_channel,lead_desc_length,event_name,inquiry_type,prefer_ver_count,prefer_ver_mean,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,transfer_agreement,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,lead_date_y,lead_date_m,lead_date_dow
15127,750000.0,nigeria,solution,NaN,25309,2022-01-24,NaN,new,enterprise,-1.0,educator,digitalshowroom,18,it digital showroom cloud,requestforquotationorpurchase,0.0,0.0,"monior/monitortv,tv,commercialtv,pc,refrigerat...",NaN,NaN,other,lgekr,9months~1year,y,0.000543,NaN,specialpurpose,NaN,441,False,2022,1,0


In [ ]:
# 'product_category' 열의 각 값을 쉼표로 분할합니다.
pre_train['product_category'] = pre_train['product_category'].str.split(',')

# explode 함수를 사용하여 각 분할된 값을 별도의 행으로 만듭니다.
pre_train = pre_train.explode('product_category')

In [ ]:
pre_train.loc[22696]

expected_budget                                   750000.0
customer_country                                     india
business_unit                                           it
com_reg_ver_win_rate                                   NaN
customer_idx                                         35326
lead_date                                       2022-05-21
customer_type                                          NaN
customer_history                                       new
enterprise                                      enterprise
historical_existing_cnt                               -1.0
customer_job                                      educator
lead_from_channel                                 facebook
lead_desc_length                                         3
event_name                                it hq oledpro fb
inquiry_type               quotationorpurchaseconsultation
prefer_ver_count                                       NaN
prefer_ver_mean                                        N

### expected_timeline : 문자열로 표현된 기간을 숫자로 대체

In [ ]:
# 기간이 문자열로 표현됨
pre_train['expected_timeline'].value_counts().iloc[:10]

expected_timeline
lessthan3months       17513
3months~6months        5111
morethanayear          3058
9months~1year          1144
6months~9months        1110
lessthan6months         108
etc.                     95
less_than_3_months       76
beingfollowedup          66
beingfollowedup.         24
Name: count, dtype: int64

In [ ]:
# 'month', 'year'를 포함하는 문자열인지 판단
def is_period(text):
  if pd.isna(text):
      return np.nan

  for keyword in ['month', 'year']:
    if keyword in text:
      return text
    else:
      continue

  return np.nan

In [ ]:
# 기간으로 표현할 수 있는 데이터만 뽑아냄
period_timeline_train = pre_train['expected_timeline'].apply(is_period).copy()
period_timeline_test = pre_test['expected_timeline'].apply(is_period).copy()

In [ ]:
def extract_numbers(text):
  # 'a', 'one', 'next'을 숫자 1로 대체
  text = re.sub(r'\ba\b', '1', text)
  text = re.sub(r'\bone\b', '1', text)
  text = re.sub(r'\bnext\b', '1', text)

  numbers = re.findall(r'\d+', text)

  cleaned_numbers = []
  for number in numbers:
    # 숫자 뒤에 'month'나 'year'가 있는지 확인하고 number list에 담아줌
    # year이면 12개월이므로 12를 곱해줌
    if re.search(r'\b{}\s*(month)s?\b'.format(number), text):
      cleaned_numbers.append(float(number))
    elif re.search(r'\b{}\s*(year)s?\b'.format(number), text):
      cleaned_numbers.append(float(number)*12)

  return text, cleaned_numbers

extract_numbers('9 months 2 years')

('9 months 2 years', [9.0, 24.0])

In [ ]:
def timeline_to_number(text):
  if pd.isna(text):
        return np.nan

  # 문자열에서 숫자 추출
  _, numbers = extract_numbers(text)
  if len(numbers) == 1:
    num = numbers[0]
    if 'less than' in text:
      return num / 2
    elif 'more than' in text: # 6개월 후로 바꿔줌
      return (6 + num)
    else:
      return num

  elif len(numbers) == 2: # 기간일 때
      num1 = numbers[0]
      num2 = numbers[1]
      return (num1 + num2) / 2

  else:
      return np.nan  # 처리할 수 없는 경우 np.nan 반환

timeline_to_number('9 months 2 years')

16.5

In [ ]:
period_timeline_train = period_timeline_train.apply(timeline_to_number)
period_timeline_test = period_timeline_test.apply(timeline_to_number)

period_mean_train = round(period_timeline_train.mean(), 3)
print(period_mean_train)

5.885


In [ ]:
# object_columns = object_columns + columns_categorized
numerical_features = list(set(pre_train.columns) - set(object_columns))
numerical_features

['com_reg_ver_win_rate',
 'ver_win_rate_x',
 'lead_desc_length',
 'is_converted',
 'customer_idx',
 'prefer_ver_count',
 'lead_owner',
 'lead_date_dow',
 'ver_win_ratio_per_bu',
 'lead_date_m',
 'lead_date_y',
 'historical_existing_cnt',
 'prefer_ver_mean']

In [ ]:
# 결측치를 train data의 평균값으로 채워줌
pre_train['expected_timeline_num'] = period_timeline_train.fillna(period_mean_train)
pre_test['expected_timeline_num'] = period_timeline_test.fillna(period_mean_train)
numerical_features.append('expected_timeline_num')

In [ ]:
pre_train['expected_timeline_num'].unique()

array([ 5.885,  4.5  ,  7.5  , 10.5  ,  3.   ,  8.   ])

In [ ]:
pre_test['expected_timeline_num'].unique()

array([ 5.885,  4.5  ,  7.5  , 10.5  ])

### 스케일링

In [ ]:
features = ['com_reg_ver_win_rate','lead_desc_length','ver_win_ratio_per_bu','historical_existing_cnt']

print(pre_train[features].min())
print(pre_train[features].max())

com_reg_ver_win_rate       0.003788
lead_desc_length           1.000000
ver_win_ratio_per_bu       0.011583
historical_existing_cnt   -1.000000
dtype: float64
com_reg_ver_win_rate          1.000000
lead_desc_length           1264.000000
ver_win_ratio_per_bu          0.285714
historical_existing_cnt    1394.000000
dtype: float64


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

pre_train[features] = scaler.fit_transform(pre_train[features]) # fit_transform
pre_test[features] = scaler.transform(pre_test[features]) # transform

In [ ]:
print(pre_train[features].min())
print(pre_train[features].max())

com_reg_ver_win_rate       0.0
lead_desc_length           0.0
ver_win_ratio_per_bu       0.0
historical_existing_cnt    0.0
dtype: float64
com_reg_ver_win_rate       1.0
lead_desc_length           1.0
ver_win_ratio_per_bu       1.0
historical_existing_cnt    1.0
dtype: float64


In [ ]:
pre_train[features].head()

,com_reg_ver_win_rate,lead_desc_length,ver_win_ratio_per_bu,historical_existing_cnt
0,NaN,0.022961,0.426761,0.003584
1,0.387925,0.035629,0.135142,0.000000
2,NaN,0.001584,NaN,0.000000
3,0.013808,0.182898,0.135142,0.000000
4,NaN,0.002375,NaN,0.000000


In [ ]:
pre_train.iloc[[1,2]]

,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,customer_job,lead_from_channel,lead_desc_length,event_name,inquiry_type,prefer_ver_count,prefer_ver_mean,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,transfer_agreement,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,lead_date_y,lead_date_m,lead_date_dow,expected_timeline_num
1,100000.0,unitedstates,id,0.387925,42490,2023-05-31,NaN,new,smb,0.0,NaN,promotionpage,0.035629,id us createboard,quotationorpurchaseconsultation,0.095436,0.067718,NaN,digitalsignageorcommercialtvs,NaN,none,lgeus,NaN,n,0.000572,0.135142,education,NaN,501,True,2023,5,2,5.885
2,100000.0,unitedstates,it,NaN,10350,2023-04-19,solutioneco-partner,new,smb,0.0,NaN,NaN,0.001584,it us cloud it,quotationorpurchaseconsultation,NaN,NaN,NaN,NaN,NaN,none,lgeus,NaN,y,NaN,NaN,NaN,NaN,375,True,2023,4,2,5.885


## 3. 모델 학습

### catboost

In [ ]:
# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
is_holdout = False
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 최적의 하이퍼파라미터 설정
#best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
models = []
best_f1_score = 0

resetIndex_train = pre_train.reset_index(drop=True)

train_x = resetIndex_train.drop(columns=['is_converted'])
train_y = resetIndex_train['is_converted'].astype(int)
train_x = train_x.fillna(0)

class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_y)

for train_index, valid_index in skf.split(train_x, train_y):
    print(train_index)
    print(valid_index)
    X_train, X_valid = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_valid = train_y[train_index], train_y[valid_index]

    # CatBoost 모델 생성 및 학습
    model = CatBoostClassifier(loss_function='Logloss', cat_features=np.where(pre_train.dtypes ==object)[0], eval_metric='F1',
                               task_type='GPU',one_hot_max_size=4,
                               random_seed=42, verbose=500, iterations = 1500, class_weights = {0:class_weights[0], 1:class_weights[1]})

    model.fit(X_train, y_train, eval_set=[(X_valid,y_valid)])

    # 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
    valid_predictions = model.predict(X_valid)

    print(valid_predictions)
    print(y_valid[:5])
    macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
    macro_f1_scores.append(macro_f1)
    models.append(model)

    if macro_f1 > best_f1_score:
        best_f1_score = macro_f1
        best_classification_model = model
        y_val = y_valid
        pred_val = valid_predictions
    if is_holdout:
        break

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)

[    0     1     2 ... 59857 59858 59860]
[    7     9    11 ... 59845 59853 59859]
Learning rate set to 0.043332
0:	learn: 0.8531371	test: 0.8735253	best: 0.8735253 (0)	total: 51ms	remaining: 1m 16s
500:	learn: 0.9601878	test: 0.9599588	best: 0.9616384 (333)	total: 23.4s	remaining: 46.7s
1000:	learn: 0.9691902	test: 0.9596691	best: 0.9616384 (333)	total: 47s	remaining: 23.4s
1499:	learn: 0.9729439	test: 0.9567816	best: 0.9616384 (333)	total: 1m 10s	remaining: 0us
bestTest = 0.9616383895
bestIteration = 333
Shrink model to first 334 iterations.
[0 0 0 ... 0 0 0]
7     0
9     0
11    0
13    0
14    0
Name: is_converted, dtype: int64
[    0     1     2 ... 59857 59858 59859]
[   10    17    21 ... 59852 59855 59860]
Learning rate set to 0.043331
0:	learn: 0.8582642	test: 0.8657863	best: 0.8657863 (0)	total: 43.7ms	remaining: 1m 5s
500:	learn: 0.9625862	test: 0.9565945	best: 0.9576587 (487)	total: 23.2s	remaining: 46.3s
1000:	learn: 0.9710548	test: 0.9583816	best: 0.9588267 (985)	total:

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[1, 0])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
get_clf_eval(y_val, pred_val)

오차행렬:
 [[  941    38]
 [  431 10562]]

정확도: 0.9608
정밀도: 0.6859
재현율: 0.9612
F1: 0.8005


## 4. 제출하기

### 테스트 데이터 예측

In [ ]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [ ]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)
x_test.head()

,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,customer_job,lead_from_channel,lead_desc_length,event_name,inquiry_type,prefer_ver_count,prefer_ver_mean,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,transfer_agreement,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,lead_date_y,lead_date_m,lead_date_dow,expected_timeline_num
0,100000.0,corporate,id,0.041010,19119,2023-07-12,endcustomer,new,smb,0.0,NaN,[premql]us_id__20230701,0.243864,id us event coccpartnersexpo,quotationorpurchaseconsultation,0.211618,0.027756,NaN,NaN,NaN,none,lgeus,NaN,NaN,0.003079,0.193276,corporate/office,others,194,2023,7,2,5.885
1,10000.0,india,as,0.143816,35859,2023-08-05,endcustomer,new,enterprise,0.0,administrator,lg.com(local),0.030879,as il lg.com i2b,quotationorpurchaseconsultation,0.057692,0.005677,vrf,NaN,NaN,manager,lgeil,lessthan3months,NaN,0.000572,0.145505,education,institute&academy,50,2023,8,5,5.885
2,30000.0,india,as,0.459491,38658,2023-08-20,endcustomer,new,smb,0.0,engineer,lg.com(local),0.059382,as il lg.com i2b,quotationorpurchaseconsultation,0.076923,0.019648,multi-split,NaN,NaN,manager,lgeil,lessthan3months,NaN,0.000060,0.426761,hospital&healthcare,hospital,9,2023,8,6,5.885
3,100000.0,egypt,as,NaN,29043,2023-06-20,endcustomer,new,enterprise,0.0,educator,facebook,0.018211,as eg facebookleadscampaign,quotationorpurchaseconsultation,NaN,NaN,etc.,NaN,NaN,director,lgeeg,3months~6months,y,NaN,NaN,NaN,NaN,51,2023,6,1,4.500
4,10000.0,india,id,0.025233,42617,2023-06-08,channelpartner,new,smb,0.0,administrator,lg.com(local),0.114014,id il lg.com,quotationorpurchaseconsultation,0.145228,0.103511,standardsignage,uh7f-hseries,65uh7f-h,ceo/founder,lgeil,lessthan3months,y,0.001183,0.139558,retail,NaN,174,2023,6,3,5.885


In [ ]:
train_x.head()

,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,customer_job,lead_from_channel,lead_desc_length,event_name,inquiry_type,prefer_ver_count,prefer_ver_mean,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline,transfer_agreement,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,lead_date_y,lead_date_m,lead_date_dow,expected_timeline_num
0,100000.0,indonesia,as,0.000000,33931,2022-10-15,end-customer,existing,enterprise,0.003584,engineer,lg.com(local),0.022961,as in lg.com i2b,quotationorpurchaseconsultation,0.076923,0.019648,vrf,0,0,director,lgein,lessthan3months,y,0.000060,0.426761,hospital&healthcare,generalhospital,870,2022,10,5,5.885
1,100000.0,unitedstates,id,0.387925,42490,2023-05-31,0,new,smb,0.000000,0,promotionpage,0.035629,id us createboard,quotationorpurchaseconsultation,0.095436,0.067718,0,digitalsignageorcommercialtvs,0,none,lgeus,0,n,0.000572,0.135142,education,0,501,2023,5,2,5.885
2,100000.0,unitedstates,it,0.000000,10350,2023-04-19,solutioneco-partner,new,smb,0.000000,0,0,0.001584,it us cloud it,quotationorpurchaseconsultation,0.000000,0.000000,0,0,0,none,lgeus,0,y,0.000000,0.000000,0,0,375,2023,4,2,5.885
3,300000.0,germany,id,0.013808,19750,2021-10-30,0,new,smb,0.000000,educator,lg.com,0.182898,id hq lg.com,quotationorpurchaseconsultation,0.095436,0.067718,oledsignage,0,0,manager,lgedg,lessthan3months,y,0.000572,0.135142,education,0,359,2021,10,5,5.885
4,100000.0,singapore,id,0.000000,45802,2023-04-11,specifier/influencer,new,smb,0.000000,marketor,lg.com(local),0.002375,id sl lg.com i2b,quotationorpurchaseconsultation,0.000000,0.000000,videowallsignage,0,0,none,lgesl,lessthan3months,y,0.000000,0.000000,0,0,198,2023,4,1,5.885


In [ ]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21341 entries, 0 to 21340
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   expected_budget          21341 non-null  float64
 1   customer_country         21268 non-null  object 
 2   business_unit            21341 non-null  object 
 3   com_reg_ver_win_rate     6688 non-null   float64
 4   customer_idx             21341 non-null  object 
 5   lead_date                21341 non-null  object 
 6   customer_type            13892 non-null  object 
 7   customer_history         21341 non-null  object 
 8   enterprise               21341 non-null  object 
 9   historical_existing_cnt  21341 non-null  float64
 10  customer_job             15505 non-null  object 
 11  lead_from_channel        20514 non-null  object 
 12  lead_desc_length         21341 non-null  float64
 13  event_name               19464 non-null  object 
 14  inquiry_type          

In [ ]:
test_pred = best_classification_model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 1) for pred in test_pred)

[0 0 1 ... 0 0 0]


2262

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [ ]:
best_classification_model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,customer_idx,40.936286
1,lead_owner,17.013769
2,lead_date_y,8.823832
3,lead_desc_length,3.614963
4,customer_type,3.388187
5,lead_date_m,2.716448
6,event_name,2.642087
7,lead_from_channel,1.919612
8,ver_win_rate_x,1.700502
9,historical_existing_cnt,1.624209


### 모델 저장 및 재검증

In [ ]:
best_classification_model.save_model('model_1330_1.cbm')

In [ ]:
model = CatBoostClassifier()
model = model.load_model('model_1330_1.cbm')

In [ ]:
for n in columns_categorized:
    pre_test[n] = pre_test[n].fillna(0)
    pre_test[n] = pre_test[n].astype('int').astype('object')

In [ ]:
# 예측에 필요한 데이터 분리
x_test = pre_test.drop(["is_converted"], axis=1)

In [ ]:
test_pred = model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 1) for pred in test_pred)

[0 0 1 ... 0 0 0]


2262